# 要約 
このJupyter Notebookは、言語モデルを使用して「20の質問」ゲームを実装することに取り組んでいます。このゲームは、プレイヤーが特定の隠された単語をできるだけ少ない質問で当てる推測ゲームです。

### 主な内容と手法:
1. **クラス設計**:
   - **Player**: プレイヤーの基本クラスで、チーム名を管理します。
   - **Questioner**: 質問者クラスで、質問を行い、ユーザーから「はい」または「いいえ」の回答を受け取ります。
   - **Guesser**: 推測者クラスで、隠された単語の推測を行います。
   - **TwentyQuestionsGame**: ゲームロジックを管理するクラスで、各ラウンド、質問、推測を管理します。
   - **TwentyQuestions**: ゲーム全体を構成し、質問リストや単語リストを管理するクラスです。

2. **データの管理**:
   - ゲーム内で使用される質問や単語は、それぞれリストとして管理されており、ランダムに選ばれたり生成されたりします。

3. **実行フロー**:
   - ゲームは20ラウンドにわたって行われ、各ラウンドごとに質問と推測が行なわれます。ゲームの結果は、質問の正答と推測に基づいて決まります。

4. **使用ライブラリ**:
   - `numpy`や`pandas`などの分析ライブラリがインポートされていますが、具体的な利用は記載されていません。

このNotebookは、「20の質問」ゲームにおけるプレイヤーのインタラクションを模倣するクラスを作成し、ゲームの進行を実装しています。ゲームは対話的で、Pythonの標準入力を利用してプレイヤーからの応答を受け付けます。

---


# 用語概説 
以下は、Jupyter Notebook内のコードに関連する専門用語の簡単な解説です。特に初心者がつまずきそうなマイナーなものや、実務を経験していないと馴染みのないものに焦点を当てています。

1. **クラス (Class)**:
   Pythonにおけるオブジェクト指向プログラミングの基本概念で、属性（データ）とメソッド（機能）をまとめたテンプレートです。ここでは、`Player`, `Questioner`, `Guesser`, `TwentyQuestionsGame`, `TwentyQuestions`などのクラスが定義されています。

2. **継承 (Inheritance)**:
   あるクラス（親クラス）から別のクラス（子クラス）が属性やメソッドを引き継ぐ機能です。`Questioner`と`Guesser`は`Player`クラスを継承しています。これにより、共通の機能を持たせつつ、それぞれの特化した機能を追加できます。

3. **ストリーム (Stream)**:
   プログラムがデータをリアルタイムで処理する際に使用される概念。ここでは、`input`関数がユーザーからの入力を受け取るためのストリームの一部として機能します。

4. **フィールド (Field)**:
   オブジェクトの中にある属性やメソッドの定義に関連する用語。ここでの`Team_name`, `hidden_word`などは、クラスのフィールド（属性）です。

5. **掛け算 (Multiplier)**:
   `player`や`question`などの要素が複数回使用されるため、`multiplier`の概念が適用される場合があります。ただし、このノートブック内では直接表現されていないため、プログラム全般の概念として理解しておくと便利です。

6. **ラウンド (Round)**:
   ゲーム内での一連の行動を指し、1回の質問と答えからなる一つのサイクルです。このコードでは、最大20ラウンドまで実施されます。

7. **隠された単語 (Hidden Word)**:
   ゲームのターゲットとなる単語で、プレイヤーは質問を通じてこの単語を当てに行きます。この概念は「20の質問」ゲーム特有のもので、隠された単語を推測するために様々な質問が使われます。

8. **カテゴリー (Category)**:
   各質問がどの領域に属するかを示すタグで、`self.questions`リスト内では各質問と共にカテゴリー情報が保存されています。質問の戦略を組織的に整理するために使用されます。

9. **乱数 (Random)**:
   疑似的にランダムな数値を生成すること。`random.choice()`関数を使って、質問や推測をランダムに選択します。このため、予測可能性を減らし、ゲームの面白みを増します。

10. **入力/出力 (I/O)**:
    プログラムが外部とデータを交換する方法を指します。ここでは、ユーザーからの入力（質問に対する答えなど）を受け取り、プログラムの結果を表示する出力を行います。

11. **データ構造 (Data Structure)**:
    データを整理し、効率的に管理するための方法です。ここではリストを使用していますが、質問や単語を格納する際のデータ構造として重要です。

こうした用語は、プログラムを理解する上で必須の知識となるため、しっかりと理解しておくことをお勧めします。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています。
# これは、kaggle/pythonのDockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、ここではいくつかの便利なパッケージをロードします。

import numpy as np # 線形代数
import pandas as pd # データ処理、CSVファイルの入出力 (例: pd.read_csv)

# 入力データファイルは読み取り専用の "../input/" ディレクトリにあります
# 例えば、以下を実行すると (実行ボタンをクリックするか、Shift + Enterを押すことで) 入力ディレクトリ内のすべてのファイルがリストされます。

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ (/kaggle/working/) に最大20GBのデータを書き込むことができ、これは「Save & Run All」を使ってバージョンを作成するときに出力として保存されます。
# 一時ファイルは /kaggle/temp/ にも書き込むことができますが、現在のセッションの外では保存されません。

In [ ]:
import random
import time

# プレイヤークラスの定義
class Player:
    def __init__(self, team_name):
        self.team_name = team_name  # チーム名の初期化

# 質問者クラスはプレイヤークラスを継承
class Questioner(Player):
    def ask_question(self, question):
        print(f"{self.team_name} が質問します: {question}")
        answer = input("答え (はい/いいえ): ").strip().lower()  # ユーザーに答えを尋ねる
        return answer == "はい"  # "はい" ならTrueを返す

# 推測者クラスもプレイヤークラスを継承
class Guesser(Player):
    def make_guess(self, guess):
        print(f"{self.team_name} の推測: {guess}")  # 推測を表示

# 20の質問ゲームのクラス
class TwentyQuestionsGame:
    def __init__(self, team1_name, team2_name, hidden_word):
        self.team1_questioner = Questioner(team1_name)  # チーム1の質問者
        self.team1_guesser = Guesser(team1_name)  # チーム1の推測者
        self.team2_questioner = Questioner(team2_name)  # チーム2の質問者
        self.team2_guesser = Guesser(team2_name)  # チーム2の推測者
        self.hidden_word = hidden_word  # 隠された単語
        self.current_round = 0  # 現在のラウンド
        self.team1_score = 0  # チーム1の得点
        self.team2_score = 0  # チーム2の得点

    def start_game(self):
        print(f"20の質問ゲームへようこそ!")
        print(f"隠された単語は: {self.hidden_word} に関連しています")
        print("さあ、プレイしましょう!\n")

    def play_round(self):
        self.current_round += 1  # ラウンドカウントを増やす
        print(f"ラウンド {self.current_round}")

        # 各チームが質問を行う
        team1_answer = self.team1_questioner.ask_question(self.generate_question())
        team2_answer = self.team2_questioner.ask_question(self.generate_question())

        # どちらかのチームが答えた場合、ゲームを終了する
        if team1_answer or team2_answer:
            self.end_game(winner=self.team1_questioner if team1_answer else self.team2_questioner)
        else:
            # 答えが見つからなかった場合、推測を行う
            self.team1_guesser.make_guess(self.generate_guess())
            self.team2_guesser.make_guess(self.generate_guess())

    def end_game(self, winner):
        # 勝者が誰かで得点を更新
        if winner == self.team1_questioner:
            self.team1_score += 1
        elif winner == self.team2_questioner:
            self.team2_score += 1

        print(f"{winner.team_name} がラウンドに勝ちました!")
        print(f"スコア: {self.team1_score} - {self.team2_score}\n")

        # ラウンド数が20未満の場合、次のラウンドを開始
        if self.current_round < 20:
            print("次のラウンドを開始します...\n")
            time.sleep(1)  # 次のラウンドを始める前に小さな遅延
            self.play_round()
        else:
            print("ゲーム終了! 最終スコア:")
            print(f"{self.team1_questioner.team_name}: {self.team1_score}")
            print(f"{self.team2_questioner.team_name}: {self.team2_score}")
            if self.team1_score > self.team2_score:
                print(f"{self.team1_questioner.team_name} がゲームに勝ちました!")
            elif self.team2_score > self.team1_score:
                print(f"{self.team2_questioner.team_name} がゲームに勝ちました!")
            else:
                print("引き分けです!")

    def generate_question(self):
        return random.choice(self.questions)["question"]  # ランダムに質問を生成

    def generate_guess(self):
        return random.choice(self.allwords)  # ランダムに推測を生成

# 20の質問用の質問リストを管理するクラス
class TwentyQuestions:
    def __init__(self):
        # 質問のリストを初期化
        self.questions = [
            {"question": "それは概念ですか、物体ですか、それとも活動ですか？", "category": "initial"},
            {"question": "それは有形のアイテムですか？", "category": "tangibility"},
            {"question": "それは主に専門家によって使用されますか？", "category": "usage"},
            {"question": "それは贅沢品と見なされますか？", "category": "value"},
            {"question": "それは通常屋外にありますか？", "category": "location"},
            {"question": "それには動く部品がありますか？", "category": "mechanics"},
            {"question": "それは現代技術の一部ですか？", "category": "era"},
            {"question": "それは車よりも大きいですか？", "category": "size"},
            {"question": "それは特定の業界で使用されますか？", "category": "industry"},
            {"question": "それは家庭で一般的に使用されますか？", "category": "common_usage"},
            {"question": "それは主にエンターテイメントに使用されますか？", "category": "function"},
            {"question": "それは美的魅力がありますか？", "category": "appearance"},
            {"question": "それは科学または研究に関連していますか？", "category": "field"},
            {"question": "それは日常生活にとって欠かせないアイテムですか？", "category": "necessity"},
            {"question": "それは通常子供によって使用されますか？", "category": "user_age"},
            {"question": "それは健康や医療に関連していますか？", "category": "field"},
            {"question": "それはほとんどのオフィスにありますか？", "category": "location"},
            {"question": "それは建設業界で使用されますか？", "category": "industry"},
            {"question": "それはコレクターアイテムと見なされますか？", "category": "value"},
            {"question": "それは料理や食事の準備に使用されますか？", "category": "usage"}
        ]
        
        # すべての単語のリスト
        self.allwords = [
            '広告', 'アガベ', 'エアコンプレッサー', 'エアコン', 'エアフィルター', 
            'エアベント', 'アラームシステム', 'アナロジー', 'アネモネ', '麻酔', 
            'アップルパイ', 'エプロン', '水族館', '大気', '講堂', 
            'バックレスト', 'バクテリア', 'バゲット', 'バルコニー', '銀行', 
            '理髪椅子', 'バーコード', 'コウモリ', 'バスマット', 'バッテリー銀行', 
            'ビーンバッグ', 'ベッドフレーム', '自転車', '自転車道', '双眼鏡', 
            '鳥かご', '鳥の種', '応援団席', 'ブラインド', 'ボードゲーム', 
            'ボビーピン', 'ボラード', '接着剤', '本棚', 'ブッケン', 
            'ボトル水', 'パンナコッタ', '休憩室', 'ブリュワリー商品', 
            'ブリーフケース', 'パンフレット', '破損したガラス', 'ブラウニー', 'バグスプレー', 
            '電球', 'バンパーステッカー', 'バンゼンバーナー', '蝶', 'キャビネット', 
            '電卓', 'カンタロープ', 'チャイルドシート', 'カード', '段ボール箱', 
            'レジ', '猫のベッド', '猫キャリア', 'カリフラワー', '天井ファン', 
            'シリアル', 'ラテ', 'シャンパングラス', 'シャンデリア', 'チーズケーキ', 
            'チェスボード', '噛むおもちゃ', 'チョコレートケーキ', 'シナモンロール', 'ぼろ布', 
            # （省略）数多くのアイテムが続く...
        ]

        self.game = TwentyQuestionsGame("チームA", "チームB", self.random_word())  # ゲームを初期化

    def random_word(self):
        return random.choice(self.allwords)  # ランダムに単語を選択

    def play_game(self):
        self.game.start_game()  # ゲームを開始
        self.game.play_round()  # ラウンドを実行

# メイン処理部
if __name__ == "__main__":
    twenty_questions = TwentyQuestions()  # ゲームのインスタンスを作成
    twenty_questions.play_game()  # ゲームをプレイ
    
    # 次の質問を取得するメソッド
    def next_question(self):
        if self.current_question_index < len(self.questions):
            question = self.questions[self.current_question_index]  # 現在の質問を取得
            return question["question"]
        else:
            return None  # 質問が終わった場合はNoneを返す

    # 回答を提供するメソッド
    def provide_answer(self, answer):
        current_question = self.questions[self.current_question_index]  # 現在の質問を取得
        category = current_question["category"]  # カテゴリーを取得
        self.categories[category] = answer  # カテゴリーに回答を保存
        self.current_question_index += 1  # 次の質問に進む

    # 単語を推測するメソッド
    def guess_word(self):
        if self.current_question_index >= len(self.questions):  # 質問が全て終わったら
            print(f"すべての質問をしました。隠された単語は '{self.hidden_word}' ですか？")
            answer = input("答え (はい/いいえ): ").lower().strip()  # ユーザーに確認
            if answer == "はい":
                print("おめでとうございます！隠された単語を正しく推測しました！")
            else:
                print(f"残念ながら、隠された単語は '{self.hidden_word}' でした。またの機会に頑張ってください！")
        else:
            print("推測する前にすべての20の質問をする必要があります。")

# ゲームを開始してプレイ
team1_name = "チームA"
team2_name = "チームB"

game = TwentyQuestionsGame(team1_name, team2_name, hidden_word="象")  # ゲームを初期化
game.start_game()  # ゲームを開始

for _ in range(20):
    game.play_round()  # 20ラウンドをプレイ